In [2]:
from tokenization import encode, decode, vocab_size, vocab
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
EMBEDDING_DIM = 32

In [4]:
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=EMBEDDING_DIM)
embedding_layer

Embedding(81, 32)

In [19]:
embedding_layer.weight

Parameter containing:
tensor([[ 0.0199,  0.4202, -1.0543,  ...,  0.0318,  0.5748,  0.3618],
        [-0.6898,  3.3488,  2.5956,  ..., -0.0289,  1.3268, -0.2817],
        [-0.2539, -0.8296, -0.1288,  ..., -1.1616, -0.9264,  0.3339],
        ...,
        [ 0.3923, -1.4268,  2.0271,  ..., -0.8077,  0.0035,  0.8254],
        [ 0.0321, -1.0122,  0.7443,  ...,  0.1071,  1.8215,  0.6387],
        [-0.3394,  0.9999,  0.2835,  ..., -0.9982,  0.2616,  0.2585]],
       requires_grad=True)

In [11]:
input_ids = encode("Bhisma is very wise.")
print(f"input_ids: {input_ids}")
print(f"input_ids: {torch.tensor(input_ids)}")
embeddings = embedding_layer(torch.tensor(input_ids))
embeddings.shape

input_ids: [4, 0, 67, 23, 30, 29, 27, 67, 23, 27, 76, 25, 61, 48, 27, 53, 67, 23, 25, 34]
input_ids: tensor([ 4,  0, 67, 23, 30, 29, 27, 67, 23, 27, 76, 25, 61, 48, 27, 53, 67, 23,
        25, 34])


torch.Size([20, 32])

In [14]:
embedding_of_B = embeddings[0]
embedding_of_B

tensor([-0.6767,  0.9284,  1.1302, -1.2293, -0.2218, -0.5815,  1.1720, -0.1322,
        -1.9772,  0.3813, -0.5317, -1.1737, -0.8880, -1.3582, -0.7713, -0.7702,
        -0.5238,  0.6970,  0.0966,  0.4947, -0.6282,  1.2888,  0.9893,  0.0439,
        -0.1704,  0.8909,  0.5912, -0.9379,  0.8538,  0.4885,  0.4097, -0.4588],
       grad_fn=<SelectBackward0>)

In [63]:
max_len = 100
d_model = EMBEDDING_DIM
pe = torch.zeros(max_len, d_model)
pe.shape

torch.Size([100, 32])

In [34]:
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
position.shape

torch.Size([100, 1])

$$\frac{1}{10000^{2i/d_{model}}}$$

In [38]:
torch.arange(0, d_model, 2).float() 

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24., 26.,
        28., 30.])

In [39]:
-math.log(10000.0) / d_model

-0.28782313662425574

In [40]:
torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)

tensor([-0.0000, -0.5756, -1.1513, -1.7269, -2.3026, -2.8782, -3.4539, -4.0295,
        -4.6052, -5.1808, -5.7565, -6.3321, -6.9078, -7.4834, -8.0590, -8.6347])

In [37]:
import math
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
div_term.shape

torch.Size([16])

In [62]:
div_term

tensor([1.0000e+00, 5.6234e-01, 3.1623e-01, 1.7783e-01, 1.0000e-01, 5.6234e-02,
        3.1623e-02, 1.7783e-02, 1.0000e-02, 5.6234e-03, 3.1623e-03, 1.7783e-03,
        1.0000e-03, 5.6234e-04, 3.1623e-04, 1.7783e-04])

In [59]:
print(position*div_term)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 5.6234e-01, 3.1623e-01,  ..., 5.6234e-04, 3.1623e-04,
         1.7783e-04],
        [2.0000e+00, 1.1247e+00, 6.3246e-01,  ..., 1.1247e-03, 6.3246e-04,
         3.5566e-04],
        ...,
        [9.7000e+01, 5.4547e+01, 3.0674e+01,  ..., 5.4547e-02, 3.0674e-02,
         1.7249e-02],
        [9.8000e+01, 5.5109e+01, 3.0990e+01,  ..., 5.5109e-02, 3.0990e-02,
         1.7427e-02],
        [9.9000e+01, 5.5672e+01, 3.1307e+01,  ..., 5.5672e-02, 3.1307e-02,
         1.7605e-02]])


In [66]:
pe[:, 0::2] = torch.sin(position * div_term)
print(pe.shape)

torch.Size([100, 32])


In [67]:
pe[:, 1::2] = torch.cos(position * div_term)
print(pe.shape)

torch.Size([100, 32])


In [68]:
pe

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  5.3317e-01,  ...,  1.0000e+00,
          1.7783e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.0213e-01,  ...,  1.0000e+00,
          3.5566e-04,  1.0000e+00],
        ...,
        [ 3.7961e-01, -9.2515e-01, -9.0865e-01,  ...,  9.9953e-01,
          1.7248e-02,  9.9985e-01],
        [-5.7338e-01, -8.1929e-01, -9.9136e-01,  ...,  9.9952e-01,
          1.7426e-02,  9.9985e-01],
        [-9.9921e-01,  3.9821e-02, -7.6875e-01,  ...,  9.9951e-01,
          1.7604e-02,  9.9985e-01]])

In [ ]:
10, 32

In [71]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        """
        Args:
            d_model: The dimension of the embeddings (32 in your case).
            max_len: The maximum sequence length your model will handle.
            dropout: Dropout probability.
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create a matrix of [max_len, d_model] representing the positional encodings
        pe = torch.zeros(max_len, d_model)
        
        # Create a vector of positions [0, 1, 2, ..., max_len-1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # Calculate the division term for the sine/cosine formulas
        # div_term = 1 / (10000 ^ (2i / d_model))
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add a batch dimension: [1, max_len, d_model]
        pe = pe.unsqueeze(0)

        # Register 'pe' as a buffer. 
        # This means it's part of the state_dict but is NOT a learnable parameter.
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Input tensor of shape (batch_size, seq_len, d_model)
        """
        # Add the positional encoding to the input embeddings
        # We slice self.pe to match the sequence length of x
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [72]:
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=EMBEDDING_DIM)
pos_encoder = PositionalEncoding(d_model=EMBEDDING_DIM)

In [76]:
text = "Bhishma is very wise"
tokens = torch.tensor(encode(text), dtype=torch.long)
print(f"tokens: {tokens}")
embeddings = embedding_layer(tokens)
embeddings.shape

tokens: tensor([ 4,  0, 67, 23,  0, 30, 29, 27, 67, 23, 27, 76, 25, 61, 48, 27, 53, 67,
        23, 25])


torch.Size([20, 32])

In [89]:
decode(tokens.tolist())

'Bhishma is very wise'

In [80]:
embeddings_with_batch = embeddings.unsqueeze(0)
embeddings_with_batch

tensor([[[-8.9629e-02, -2.3094e-02,  1.3975e+00,  3.7862e-01, -3.4076e-01,
           1.4400e+00, -1.1908e+00,  7.8693e-01, -5.1956e-02,  3.8565e-01,
           2.4341e-01,  4.2516e-02,  4.9360e-01, -8.1355e-02, -8.8898e-01,
           1.6459e+00,  4.5596e-01,  9.0416e-02,  6.1574e-01,  1.4290e+00,
          -3.0128e-01, -7.4445e-01,  4.8218e-01, -1.4849e+00,  5.5940e-01,
          -1.2439e-01,  1.3801e-01,  4.8013e-01, -8.8110e-01, -1.9698e-01,
           9.3190e-02, -7.7100e-01],
         [-2.0770e-02,  9.9981e-01,  8.4547e-01, -1.2986e+00, -3.4438e-01,
           1.6602e+00, -6.6720e-01,  1.6941e-01, -9.8163e-01,  2.8308e-01,
          -1.0780e+00, -1.1606e-01,  1.7840e-01,  5.5293e-01,  1.8219e-01,
           7.6230e-01, -9.9815e-01,  3.6888e-01,  1.7049e-01,  8.2004e-01,
          -5.3396e-01, -3.7870e-01,  6.1093e-01, -1.3668e+00, -6.7983e-01,
           8.0038e-01,  5.6062e-01,  6.7685e-01, -2.0585e-01, -1.1079e+00,
          -1.2637e+00,  7.9468e-01],
         [ 1.0372e+00,  5.

In [81]:
x = embeddings_with_batch * math.sqrt(EMBEDDING_DIM)

In [84]:
x

tensor([[[-5.0702e-01, -1.3064e-01,  7.9056e+00,  2.1418e+00, -1.9276e+00,
           8.1461e+00, -6.7363e+00,  4.4516e+00, -2.9391e-01,  2.1816e+00,
           1.3769e+00,  2.4051e-01,  2.7922e+00, -4.6021e-01, -5.0288e+00,
           9.3103e+00,  2.5793e+00,  5.1147e-01,  3.4831e+00,  8.0836e+00,
          -1.7043e+00, -4.2113e+00,  2.7276e+00, -8.4000e+00,  3.1645e+00,
          -7.0366e-01,  7.8069e-01,  2.7160e+00, -4.9843e+00, -1.1143e+00,
           5.2716e-01, -4.3614e+00],
         [-1.1749e-01,  5.6558e+00,  4.7827e+00, -7.3460e+00, -1.9481e+00,
           9.3914e+00, -3.7743e+00,  9.5835e-01, -5.5529e+00,  1.6014e+00,
          -6.0983e+00, -6.5655e-01,  1.0092e+00,  3.1278e+00,  1.0306e+00,
           4.3122e+00, -5.6464e+00,  2.0867e+00,  9.6445e-01,  4.6388e+00,
          -3.0205e+00, -2.1422e+00,  3.4560e+00, -7.7318e+00, -3.8457e+00,
           4.5276e+00,  3.1713e+00,  3.8288e+00, -1.1645e+00, -6.2673e+00,
          -7.1486e+00,  4.4954e+00],
         [ 5.8672e+00,  3.

In [85]:
x = pos_encoder(x)

In [87]:
x.shape

torch.Size([1, 20, 32])

In [ ]:
# --- Your Configuration ---
EMBEDDING_DIM = 32
vocab_size = 81
MAX_SEQUENCE_LENGTH = 50 # Example: max number of tokens in a sentence

# --- The Layers ---
# 1. Your existing embedding layer
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=EMBEDDING_DIM)

# 2. The new positional encoding layer
pos_encoder = PositionalEncoding(d_model=EMBEDDING_DIM, max_len=MAX_SEQUENCE_LENGTH)

# --- The Forward Pass ---
# Create a dummy input (Batch Size = 2, Sequence Length = 10)
input_indices = torch.randint(0, vocab_size, (2, 10))

# Step 1: Get Embeddings [2, 10, 32]
# It is common practice to scale embeddings by sqrt(d_model) before adding PE
x = embedding_layer(input_indices) * math.sqrt(EMBEDDING_DIM)

# Step 2: Add Positional Encoding
x = pos_encoder(x)

print(f"Output shape: {x.shape}") 
# Output: torch.Size([2, 10, 32])